#**Toy Example: Manifold Projections**

![notebook-result-illustration](https://raw.githubusercontent.com/swufung/AdversarialProjections/master/Toy-Examples/toy-manifold-illustration.PNG)

*(The above is a screenshot from our [paper](https://arxiv.org/pdf/2008.02200.pdf). All data plotted here were generated from running this notebook.)*

</br>

**Created by:** Howard Heaton, Samy Wu Fung, Alex Tong Lin

**Date Created:** May 2020

**Date Updated:** November 11, 2020

_Note_: Through the menu you can save a copy of this code (File --> Save a Copy) and run it online through your own Google account (or run it locally on your computer if you have Jupyter set up).


**Overview**

This notebook has been created to provide a simple example for practitioners to learn a new tool for solving inverse problems. Here the core task is recover a signal $u^\star$ from a collection of noisy measurements. One approach is to design a sort of "black box" neural network that we feed in measurements and train to (hopefully) be able to reliably reconstruct signals. Another approach is to create an analytic model and assume $u^\star$ is the solution to the optimization problem

$$ \min_{u\in\mathbb{R}^n} H(u) + J(u),$$

where $H$ is the _fidelity term_ that measures compliance with measurements and $J$ is the _regularizer_ that encodes prior knowledge about typically signals $u^\star$. In classical optimization, $J$ is chosen by hand, yielding results that are now often subpar compared to state-of-the-art machine learning. However, the classic methods possess solid theoretical justification and robustness, properties machine learning approaches often lack.

The new tool we illustrate in this notebook is how to leverage available data to construct a data-driven $J$ that fuses the strengths of machine learning with the theory of classic optimization. A key ingredient in this approach is to apply the common assumption that most signals in practice exhibit redudancies (e.g., pictures  often have many pixel values that are correlated to each other), and can therefore be represent more efficiently in some lower dimensional space. This is to say that each $u^\star$ is contained in a manifold $\mathcal{M}$ of intrinsically low dimension. Ideally, we would recover a signal estimate in $\mathcal{M}$.
This requires solving a constrained optimization problem such as

$$ \min_{u\in\mathbb{R}^n} H(u) + \delta_{\mathcal{M}}(u) = \min_{u\in\mathcal{M}} H(u).$$

To solve this problem, optimization algorithms will iteratively project estimates on $\mathcal{M}$. Unfortunately, if we only have a few samples of true signals $u^\star \in \mathcal{M}$, we are unlikely to find a closed form expression for this. However, our [paper](https://arxiv.org/pdf/2008.02200.pdf) introduces the first (to our knowledge) approach to rigorously accomplish this task, illustrated on a toy example in this notebook.

To achieve our goal of simplicity, we use a 2D example and shorten the code as much as possible (e.g., by removing the use of tensor dataloaders for batching). The notebook first consists of creating data (sampling from the manifold and generating samples to be projected onto the manifold). Then we define our neural network architecture and train the network to project the generated examples onto an approximation of the manifold. We say _approximation_ since we use only a discrete sampling of the manifold for learning and a finite number of iterations (as this resembles practical situations). After training, we use these projections to solve a basic optimization problem involving least squares. There we illustrate how projected gradient (using an analytic formula for the true manifold) compares to our adversarially projected gradient method. The only distinction between the two methods is the operator used to project onto the manifold. As we will see, the learned projection generalizes surprisingly well for the sparsely sampled manifold.


**Data Creation**

We first generate samples from true manifold $\mathcal{M}$ given by the upper half circle of radius $0.75$ centered at $(2,0)$  so that each sample $u^\star \in \mathcal{M}$ takes the form

$u^\star =\left[\begin{array}{c} 2 \\ 0 \end{array}\right] + \dfrac{3}{4} \left[\begin{array}{c} \cos(\theta) \\ \sin(\theta) \end{array}\right]$ with $\theta \sim \mbox{Uniform}[0,\pi]$.

We will restrict our interest of use for the projection operator $P_{\mathcal{M}}$ to the region $[0,3]\times [-0.5,1.5]\subset \mathbb{R}^2$. 
So, we generate random samples in $\mathbb{R}^2$ to project onto $\mathcal{M}$ using

$u^1 = \left[\begin{array}{c} \xi_1 \\ \xi_2 \end{array}\right]$
with
$\xi_1 \sim \mbox{Uniform}[0,3]$ 
and 
$\xi_2 \sim \mbox{Uniform}[-0.5,1.5]$.

Running the following code cell creates the numpy arrays ```u_gen``` and ```u_true```, and then plots both of these in the plane.

_Note_: We believe it is helpful to include a small amount of Gaussian noise to the samples in $u^1$. This ensures the distance function estimate is reasonable within a neighborhood of the points, and has been observed to yield improved results in our experiments.

In [ ]:
import matplotlib
import os
import argparse
import torch
import torch.nn              as nn
import torch.optim           as optim
import matplotlib.pyplot     as plt
import numpy                 as np
from   mpl_toolkits.axes_grid1 import make_axes_locatable
from   torch.utils.data        import Dataset, TensorDataset, DataLoader
import time

seed = 42
torch.manual_seed(seed)
device = "cpu" 

#-----------------------------------------------------------------
# Sampling and Batch Sizes
#-----------------------------------------------------------------
n_samples_man = 40  
n_samples     = 500 
#-----------------------------------------------------------------
# Manifold Data
#-----------------------------------------------------------------
u_true = torch.ones(n_samples_man, 2).to(device)
for i in range(n_samples_man):
    theta = torch.rand(1)
    r     = 1.5 
    u_true[i,0] = 2 + 0.5 * r * np.cos( 3.1415 * theta)
    u_true[i,1] = 0.5 * r * np.sin( 3.1415 * theta)
#-----------------------------------------------------------------
# Measurement Data
#-----------------------------------------------------------------
u_gen       = 3 * torch.rand(n_samples, 2).to(device)  
u_gen[:,1] -= 0.5
u_gen[:,0] += 0
u1          = u_gen.to(device)
#-----------------------------------------------------------------
# Plot Manifold Samples
#-----------------------------------------------------------------
x_min = 0
x_max = 3
y_min = -0.5
y_max = 2.5


def plot_distributions(dist1, dist2, index):
  fig = plt.figure()
  plt.scatter(dist1[:,0],dist1[:,1], c='r', alpha=0.3)
  plt.scatter(dist2[:,0],dist2[:,1], c='b', alpha=0.3)
  plt.ylim(y_min, y_max);
  plt.xlim(x_min, x_max)
  plt.legend(['manifold samples', 'generated samples'], loc='upper left',)
  title_str = 'Manifold Samples' + str(index)
  plt.title(title_str)
  plt.show()
  plt.close(fig)
  plt.close(plt.gcf())

plot_distributions(u_true, u_gen, 1)

def get_prefix(ctr): # Prefix for saving file names
    if ctr < 10:
        return '00000'
    elif ctr < 100:
        return '0000'
    elif ctr < 1000:
        return '000'
    elif ctr < 10000:
        return '00'
    elif ctr < 100000:
        return '0'
    else:
        return '' 

filename = './manifold.csv'
with open(filename, 'w') as f: 
  f.write('a,b\n')
  for file_ctr in range(u_true.shape[0]):     
    f.write('%0.5e,%0.5e\n' % (u_true[file_ctr, 0], u_true[file_ctr, 1]))   

filename = './p_' + get_prefix(1) + str(1) + '.csv'
with open(filename, 'w') as f: 
  f.write('a,b\n')
  for file_ctr in range(u_gen.shape[0]):     
    f.write('%0.5e,%0.5e\n' % (u_gen[file_ctr, 0], u_gen[file_ctr, 1])) 

**Network Definition**

The network structure used follows the ideas from the [GroupSort paper](http://proceedings.mlr.press/v97/anil19a/anil19a.pdf), which we highly recommend reading. In short, our neural net structure is 1-Lipschitz with respect to its inputs and (effectively) gradient norm preserving with respect to the training weights, which yields wonderful stability and the ability to approximate any 1-Lipschitz function arbitrarily well (with a sufficiently large network). In addition, we use a convex combination with the identity operator during feed forward operations, which acts similarly to a [ResNet idea](https://arxiv.org/pdf/1805.07477.pdf) and provides further stability (allowing us to easily train deep networks). Choosing $\beta < 1$ is not necessary in our simple problem. But, we note this still works fine (e.g., using $\beta=0.75$ gives roughly the same final result) and may find good use on other problems.

In [ ]:
class distance_estimator(nn.Module):
    def __init__(self, n_feats, n_hid_params, hidden_layers, n_projs, device, beta=0.5):
        super().__init__()        
        self.hidden_layers = hidden_layers  # number of hidden layers in the network
        self.n_projs       = n_projs        # number of projections to use for weights onto Steiffel manifold
        self.beta          = beta           # scalar in (0,1) for stabilizing feed forward operations
        
        # Intialize initial, middle, and final layers
        self.fc_one = torch.nn.Linear(n_feats, n_hid_params, bias=True )
        self.fc_mid = nn.ModuleList([torch.nn.Linear(n_hid_params, n_hid_params, bias=True ) for i in range(self.hidden_layers)])
        self.fc_fin = torch.nn.Linear(n_hid_params, 1, bias=True )

        # Normalize weights (helps ensure stability with learning rate)
        self.fc_one.weight = nn.Parameter(self.fc_one.weight/torch.norm(self.fc_one.weight))
        for i in range(self.hidden_layers):
            self.fc_mid.weight = nn.Parameter(self.fc_mid[i].weight/torch.norm(self.fc_mid[i].weight))          
        self.fc_fin.weight = nn.Parameter(self.fc_fin.weight/torch.norm(self.fc_fin.weight))
                        
    def forward(self, u):     
        u = self.fc_one(u).sort(1)[0]                             # Apply first layer affine mapping
        for i in range(self.hidden_layers):                       # Loop for each hidden layer                     
          u = u + self.beta * (self.fc_mid[i](u).sort(1)[0] - u)  # Convex combo of u and sort(W*u+b)
        u = self.fc_fin(u)                                        # Final layer is scalar (no need to sort)        
        J = torch.abs(u)        
        return J
                   
    
    def project_weights(self):
        self.fc_one.weight.data = self.proj_Stiefel(self.fc_one.weight.data, self.n_projs)
        for i in range(self.hidden_layers):
            self.fc_mid[i].weight.data = self.proj_Stiefel(self.fc_mid[i].weight.data, self.n_projs)
        self.fc_fin.weight.data = self.proj_Stiefel(self.fc_fin.weight.data, self.n_projs)           
            
    def proj_Stiefel(self, Ak, proj_iters):  # Project to closest orthonormal matrix
        n = Ak.shape[1]
        I = torch.eye(n)
        for k in range(proj_iters):
            Qk = I - Ak.permute(1, 0).matmul(Ak)
            Ak = Ak.matmul(I + 0.5 * Qk)
        return Ak

**Training Setup**

Training consists of learning a projection operator $P_{\mathcal{M}}$. This is an _intensive_ process. Once training is complete, we obtain a network $\phi:\mathbb{R}^2\rightarrow\mathbb{R}^2$ such that (hopefully)

$ \phi(u) \approx P_{\mathcal{M}}(u), \ \ \ \mbox{for all} \ u \in \mathcal{M}.$

Here

$\phi(u) = (T_K\circ T_{K-1}\circ\cdots \circ T_1)(u)$

where

$T_k(u) = (1-\beta) u + \beta \sigma (W^ku + c^k)$, for $k\in[K],$

$\sigma$ is the sorting operation, and $W^k$ and $c^k$ are a matrix and vector, respectively.

In [ ]:
n_hid_params   = 10    # Parameters in each hidden layer (10)
n_hid_layers   = 10    # Total hidden layers (10)
n_projs        = 2     # Number of projections to Stiefel manifold (2)
beta           = 1.0   # Relaxation parameter in network feed forward operations (1.0)
n_feats        = 2     # Our signals are in R^2
netD           = distance_estimator(n_feats, n_hid_params, n_hid_layers, n_projs, device, beta=beta)
max_epochs     = int(1e7)
learning_rate  = 2e-3     # (2e-3)
optimizer      = optim.SGD(netD.parameters(), lr=learning_rate)
drop_freq      = 1.0e5    # Frequency for decaying step sizes
decay_rate     = 1.0      # Step size decay proportion
prog_threshold = 1.0e-4   # Determines when to perform generator update
eta_threshold  = 0.75     # Minimum value of eta allowed to perform generator update
man_thresh     = 1.0e-2
n_steps        = 20
hist_dist      = np.zeros(n_steps)
hist_eta       = np.zeros(n_steps)
checkpt_path   = './models/'

def gamma(k): # Relaxation parameter used for Halpern updates
    return (1 + k) ** (-1)  

**Training**

Here we are trying the maximize the difference between our discriminator's values on the true data and the generated data. It is important to note that we train using a parameter $\eta$ described in the preprint. In short, this $\eta \in [0,1]$ gives an indication of the proportion of samples that we are able to detect outside the manifold. So, for example, $\eta = 0.34$ can be interpret to mean that our network detects that $34\%$ of the points are outside the manifold. If the samples are entirely outside the manifold, then we would ideally obtain $\eta\approx 1.0$ during training. So, as training progresses, the $\eta$ parameter will increase and we can use this to determine how "close" we are to obtaining our distance function $d_{\mathcal{M}}$ estimate. Moreover, because $\eta$ is restricted to the interval $[0,1]$, we can use the same stopping criterion for *all* the training, i.e., this is an absolute metric. (This is incredibly convenient since it is difficult to know how close one is to solving the problem looking only at the distance estimates...)

Note: The fact that the network is 1-Lipschitz makes is so we can have some insight into the choice of step-size to use.

In [ ]:
#--------------------------------------
#  Initialize Training
#--------------------------------------
k        = 1  # defines the index for {u^k}
ave_diff = 0
ctr      = 1  # inner loop for solving the k-th minimization problem
eta      = 0.0
dist_est = 0.0
dist_man = 0.0
loss     = 0.0
eta_prev = 0.0
uk       = u_gen

print(netD) 
#------------------------------------------------------------
# Identify if stopping criteria are met for current step
#------------------------------------------------------------
def stop_crit_met(epoch, ave_diff, eta, prog_thresh, eta_thresh, ctr, k, dist_man):    
    if epoch%1000 != 0:           # Only update once every 100 epochs
        return False
    elif ctr > 100000:
        return True 
    elif ave_diff > prog_thresh: # Require estimate progress to be small (close to optimal)
        return False
    elif ave_diff == 0:          # Require their to be forward progress 
        return False    
    elif dist_man > man_thresh:
        return False
    elif eta < eta_thresh:
        return False
    return True
#------------------
# Training
#------------------
for epoch in range(1, max_epochs + 1):
    if k < n_steps:
        t0        = time.time()               
        optimizer.zero_grad()  # Initialize gradient to zero
        err_real     = torch.mean(netD(u_true))
        # err_fake     = torch.mean(netD(uk))

        # To artifically inflate P^k, we can include noise (following line)
        err_fake     = torch.mean(netD(uk + 0.02 * torch.randn(uk.shape)))

        err          = err_real - err_fake
        soft_const   = 50 * torch.mean(netD(u_true) ** 2) # tau = 50, p = 2
        loss         = err + soft_const
        dist_est     = -0.001 * err.detach() + 0.999 * dist_est            
        loss.backward()
        optimizer.step()  
        dist_man = 0.01 * err_real.detach().numpy() + 0.99 * dist_man
        netD.project_weights()
        uk.requires_grad_(True)
        Jout   = netD(uk)
        nablaJ = torch.autograd.grad(outputs=Jout, inputs=uk, # take derivative w.r.t only inputs
                                  grad_outputs=torch.ones(Jout.size()).to(device), only_inputs=True)[0]        
        eta      = 0.001 * (torch.norm(nablaJ, p=2, dim=1) ** 2).mean().cpu().numpy() + 0.999 * eta
        ave_diff = 0.001 * np.maximum(0, eta - eta_prev) + 0.999 * ave_diff
        ctr += 1        
        if ctr%100 == 0:
            eta_prev = eta
      
        if stop_crit_met(epoch, ave_diff, eta, prog_threshold, eta_threshold, ctr, k, dist_man):
          #------------------------------------
          # Update distribution estimate uk
          #------------------------------------                
          beta = dist_est
          uk.requires_grad_(True)
          Jout   = netD(uk)
          nablaJ = torch.autograd.grad(outputs=Jout, inputs=uk,   # Compute derivative w.r.t inputs
                                    grad_outputs=torch.ones(Jout.size()).to(device), only_inputs=True)[0]                                                   
          diag  = 0.25 * (beta + Jout)
          uk    = gamma(k) * u1 + (1-gamma(k)) * (uk - diag * nablaJ)  
          uk    = uk.detach()
          k    += 1 # update outer iteration number
          ctr   = 0 # Reset counter for stopping critera  
          #--------------------------
          # Save weights/step sizes
          #--------------------------
          state = {
            'epoch': epoch,
            'eta': eta,
            'learning_rate': optimizer.param_groups[0]['lr'],
            'state_dict': netD.state_dict(),
            'beta' : beta,
            'ave_true': err_real.detach().cpu().numpy(),
          }         
          if not os.path.exists(os.path.dirname(checkpt_path)):
              print("created path: ", checkpt_path)
              os.makedirs(os.path.dirname(checkpt_path))
          save_checkpt_str = checkpt_path + 'step_' + get_prefix(k) + str(k) + '.pth'          
          torch.save(state, save_checkpt_str) 
          print('Saved Checkpoint:' + save_checkpt_str) 
        #------------------------
        # Update drop frequency
        #------------------------
        #if ctr % drop_freq == 0 and ctr > 0:
        #    optimizer.param_groups[0]['lr'] *= decay_rate        
        
        #------------------------
        # Output training stats
        #------------------------
        if epoch % 1000 == 0:
            print('[%d: %d/%d] d(uk): %.3e, d(man): %.3e, eta = %.3f, eta-diff = %0.3e, lr %.3e, timer %0.2f'
                % (k, epoch, max_epochs, dist_est, dist_man, eta, ave_diff, optimizer.param_groups[0]['lr'], time.time() - t0))
        #------------------------
        # Plot training data
        #------------------------
        if ctr == 0:
            plot_distributions(u_true.detach(), uk.detach(), k)        
        # ------------------------
        # Evaluate J on 2D grid
        # ------------------------        
        if ctr % 1000 == 0:
            fig = plt.figure()
            a = torch.linspace(0, 3, 200)
            b = torch.linspace(2.5, -0.5, 200)
            gridPts   = torch.stack(torch.meshgrid(a, b)).to(device)
            gridShape = gridPts.shape[1:]
            gridPts   = gridPts.reshape(2, -1).t()
            disc_grid = netD(gridPts)
            disc_grid = disc_grid.detach().cpu()
            
            plt.imshow(disc_grid.reshape(gridShape).t())
            plt.clim(0,2.5)
            plt.colorbar()
            J_title_str = 'Manifold Distance Function Landscape:' + str(k)
            fig.savefig('distance_landscape_' + get_prefix(epoch+1) + str(epoch+1) + '.png')
            
            plt.title(J_title_str)
            plt.close(fig)
            plt.close(plt.gcf())

# Heat map
The following snippet of code is used for creating a heat map type display of the distance function $d_{\mathcal{M}}$ landscape (later compiled in LaTex for Figure 3b at the top o).

In [ ]:
#------------------------------------------------------------------
# Save J_{theta}^1 heatmap data for Figure 3c (compiled in Latex)
#------------------------------------------------------------------
def heat_map(index):
    netD        = distance_estimator(n_feats, n_hid_params, n_hid_layers, n_projs, device, beta=1.0);
    load_path   = './models/step_' + get_prefix(index) + str(index) + '.pth'
    checkpt     = torch.load(load_path, map_location="cpu")
    netD.load_state_dict(checkpt["state_dict"])
       
    filename = './heat_map.txt'
    img_size = 64
    with open(filename, 'w') as f:    
      heat_map = np.zeros((img_size, img_size))
      for i in range(img_size):
        f.write('{')
        for j in range(img_size):
          x         = x_min + j * (x_max - x_min) / img_size
          y         = y_max + i * (y_min - y_max) / img_size
          point     = np.array([[x,y]])
          point_ten = torch.from_numpy(point).float()
          dist      = 100 / 2.5 * netD(point_ten).detach().float()
          if j == (img_size-1):
            f.write('%0.1f' % (dist))
          else:
            f.write('%0.1f,' % (dist))
        if i < img_size-1:
          f.write('},\n')
        else:
          f.write('},')        
      print('Saved heat map.')

heat_map(2)


# Application of Adversarial Projection to Toy Problem


**Toy Example Problem**

We consider the optimization problem

$\displaystyle \min_{z\in \mathbb{R}^2} H(z) + \delta_{\mathcal{M}}(z) = \min_{z \in \mathcal{M}} H(z),$

where $A = [1\ \  2]$ and $b = 2$, and

$H(u) := \dfrac{1}{2}\|Au-b\|_2^2,$


The iterative method we use to solve this problem is a relaxed version of projected gradient. For $\tau \in (0,1)$ and $\kappa \in (0, 2/\mbox{Lip}(H))$, we use relaxed projected gradient updates of the form


$\ \ \ z^{t+1} = (1-\tau)\cdot z^t + \tau\cdot P_{\mathcal{M}}(z^t - \alpha \nabla H(z^t))$.

</br>

Here we load the model parameters stored during our training to test our model on the problem above. We include two variations. One plot is of the trajectory using an analytic formula for the projection onto the manifold. The other uses our learned version, called adversarial projection. Note this could be implemented more efficiently, but the approach here is used to facilitate understanding of the process.

In [ ]:
def proj_M_L2O(uk):
    uk      = torch.from_numpy(uk)
    u1      = uk  # create anchor point
    n_steps = 20  # same number of steps as in training
    netD    = distance_estimator(n_feats, n_hid_params, n_hid_layers, n_projs, device, beta=1.0);
    #---------------------------------------------------------------------------
    # Begin push forward
    #---------------------------------------------------------------------------
    for k in range(2,n_steps+1):   
        #-----------------------------------------------------------------------
        # Load network state:
        #-----------------------------------------------------------------------
        resume_path = './models/step_' + get_prefix(k) + str(k) + '.pth'
        checkpt     = torch.load(resume_path, map_location="cpu")
        netD.load_state_dict(checkpt["state_dict"])
        #-----------------------------------------------------------------------
        # Compute J(u_true), J(u_gen), eta
        #-----------------------------------------------------------------------
        uk.requires_grad_(True)
        Jtrue = netD(u_true).mean()
        Jgen  = netD(uk.float()).mean()
        # take derivative w.r.t only inputs
        nablaJ = torch.autograd.grad(outputs=Jgen, inputs=uk,
                                  grad_outputs=torch.ones(Jgen.size()).to("cpu"), only_inputs=True)[0]
        if k > 1: # Update u^k --> u^(k+1)
            beta  = checkpt["beta"]
            diag  = 0.25 * (beta + Jgen)
            uk    = gamma(k) * u1 + (1-gamma(k)) * (uk - diag * nablaJ)
    proj = uk.detach().numpy()
    return np.transpose(proj)
#-------------------------------------------------------------------------------
# Define the fixed quantities for the toy example
#-------------------------------------------------------------------------------
A  = np.array([[1, 2]])
z1 = np.array([[1.0], [2.0]])
b  = np.array([[2.25]])
c  = np.array([[2.0], [0.0]])
L  = np.linalg.norm(np.matmul(np.transpose(A),A))
#-------------------------------------------------------------------------------
# Algorithm Parameters
#-------------------------------------------------------------------------------
tau     = 0.2
alpha   = 1.0 / L 
n_iters = 20
#-------------------------------------------------------------------------------
# Arrays for Plotting Results
#-------------------------------------------------------------------------------
trajL2O        = np.zeros((n_iters,2))
trajRPG        = np.zeros((n_iters,2))
feasible_set1  = np.zeros((100, 2))
feasible_set2  = np.zeros((100, 2))
manifold       = np.zeros((100, 2))
#-------------------------------------------------------------------------------
# Algorithm Operators
#-------------------------------------------------------------------------------
def grad_H(u): # gradient of least squares
    return np.matmul(np.transpose(A), np.matmul(A,u) - b)
def proj_M(u): # projection onto M
    return c + 0.75*(u - c) / np.linalg.norm(u - c)
#-------------------------------------------------------------------------------
# Analytic Projected Gradient
#-------------------------------------------------------------------------------
zt = z1
trajRPG[0, :] = np.transpose(zt)
for t in range(n_iters - 1):
    yt = zt - alpha * grad_H(zt)
    zt = (1 - tau) * zt + tau * proj_M(yt)
    trajRPG[t+1, :] = np.transpose(zt)
#-------------------------------------------------------------------------------
# Adversarially Projected Gradient
#-------------------------------------------------------------------------------
zt = z1
trajL2O[0, :] = np.transpose(zt)
for t in range(n_iters - 1):
    yt = zt - alpha * grad_H(zt)
    zt = (1 - tau) * zt + tau * proj_M_L2O(np.transpose(yt))
    trajL2O[t+1, :] = np.transpose(zt)
    if t%5 == 0:
        print('Completed Step ', t)
#-------------------------------------------------------------------------------
# Create Feasible Set and Manifold
#-------------------------------------------------------------------------------
for i in range(100):
    temp = (1 - i / 100) * np.array([[0],[b[0,0]/2]]) + (i / 100.0) * np.array([[b[0,0]],[0]])
    feasible_set1[i, :] = np.transpose(temp)
    #temp = (1 - i / 100) * np.array([[0],[b[1,0]]]) + (i / 100.0) * np.array([[b[1,0]],[0]])
    #feasible_set2[i, :] = np.transpose(temp)

    temp = np.array([[2],[0]]) + 0.75 * np.array([[np.cos(i * 3.14/100)],[np.sin(i * 3.14/ 100)]])
    manifold[i, :]  = np.transpose(temp)

print('Iterations Complete.')

filename = './rpg_adversarial.csv'
with open(filename, 'w') as f: 
  f.write('a,b\n')
  for file_ctr in range(trajL2O.shape[0]):     
    f.write('%0.5e,%0.5e\n' % (trajL2O[file_ctr, 0], trajL2O[file_ctr, 1])) 

filename = './rpg_analytic.csv'
with open(filename, 'w') as f: 
  f.write('a,b\n')
  for file_ctr in range(trajRPG.shape[0]):     
    f.write('%0.5e,%0.5e\n' % (trajRPG[file_ctr, 0], trajRPG[file_ctr, 1]))     

**Plotting Example Results**

Below we output results of applying the projection scheme to our toy problem (alongside trajectories for the analytic projection).

In [ ]:
#-----------------------------------------------------------------
# Create Plots
#-----------------------------------------------------------------
fig = plt.figure(num=None, figsize=(8, 8), dpi=80, facecolor='w', edgecolor='k')
plt.plot(feasible_set1[:,0],feasible_set1[:,1], c='g', alpha=0.75, linewidth=4,)
plt.plot(manifold[:,0],manifold[:,1], c='r', alpha=0.3, linewidth=4,)
plt.scatter(u_true[:,0].detach(), u_true[:,1].detach(), c='r', alpha=0.5)
plt.plot(trajRPG[:,0], trajRPG[:,1], c='b', alpha=0.9)
plt.plot(trajL2O[:,0], trajL2O[:,1], c='m', alpha=0.9)

plt.ylim(-0.5, 2.5);
plt.xlim(-0, 3)
plt.legend(['feasible set', 'manifold',  'proj grad', 'adv proj grad', 'manifold samples', ], loc='upper right',)
title_str = 'Trajectories over time'
plt.title(title_str)
plt.show()
plt.close(fig)